# Transfer Learning with Tensorflow Part 1: Feature Extraction
Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem.
There are two main benefites
* 1. Can leverage an existing neural network architecture proven to work on problems similar to our own.


* 2. Can leverage a working neural network architecture which has already learned patterns on similar data to  our own, then we can adapt those patterns to our own data.

In [1]:
!pip install tensorflow
import tensorflow as tf
# from google.colab import files
# files.download('/content/drive/MyDrive/Colab Notebooks/04-transfer-learning-in-tensorflow-part-1-feature-extraction.ipynb')

!nvidia-smi

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/bin/bash: line 1: nvidia-smi: command not found


## Download and become one with data


In [ ]:
# Get data (10% of 10 food classes from food101)
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2025-10-13 09:23:29--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.69.207, 64.233.181.207, 173.194.193.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.69.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   180MB/s    in 0.9s    

2025-10-13 09:23:30 (180 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
import zipfile
with zipfile.ZipFile("10_food_classes_10_percent.zip") as ziped_ref:
  ziped_ref.extractall()


In [ ]:
!ls 10_food_classes_10_percent/train/hamburger

1010140.jpg  1670180.jpg  2256750.jpg  3088660.jpg  3796218.jpg  583603.jpg
1036215.jpg  1709710.jpg  2278946.jpg  3101158.jpg  3805565.jpg  604162.jpg
1038553.jpg  1761259.jpg  248136.jpg   3125905.jpg  3878886.jpg  607735.jpg
1086335.jpg  1802721.jpg  2534877.jpg  3140576.jpg  3912017.jpg  699176.jpg
1122669.jpg  1818289.jpg  2565475.jpg  3156598.jpg  396083.jpg	 73340.jpg
1181766.jpg  1821268.jpg  2671820.jpg  3312172.jpg  419342.jpg	 73783.jpg
1228951.jpg  1822493.jpg  2722745.jpg  3346138.jpg  423924.jpg	 76817.jpg
124124.jpg   1828545.jpg  2797268.jpg  3354260.jpg  446192.jpg	 816775.jpg
1301947.jpg  1879189.jpg  29455.jpg    3486079.jpg  460108.jpg	 91789.jpg
1335771.jpg  1903326.jpg  2953924.jpg  3488531.jpg  477600.jpg	 985020.jpg
153931.jpg   2154348.jpg  2988095.jpg  3530013.jpg  509480.jpg
1585333.jpg  2207445.jpg  3069250.jpg  3583066.jpg  554810.jpg
1635356.jpg  2223416.jpg  3083927.jpg  379510.jpg   577673.jpg


In [ ]:
import os
import pathlib

for dirpath, dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print(f"there are {len(dirnames)} directories and {len(filenames)} images in {dirpath}")

there are 2 directories and 0 images in 10_food_classes_10_percent
there are 10 directories and 0 images in 10_food_classes_10_percent/train
there are 0 directories and 75 images in 10_food_classes_10_percent/train/sushi
there are 0 directories and 75 images in 10_food_classes_10_percent/train/hamburger
there are 0 directories and 75 images in 10_food_classes_10_percent/train/steak
there are 0 directories and 75 images in 10_food_classes_10_percent/train/ice_cream
there are 0 directories and 75 images in 10_food_classes_10_percent/train/chicken_wings
there are 0 directories and 75 images in 10_food_classes_10_percent/train/grilled_salmon
there are 0 directories and 75 images in 10_food_classes_10_percent/train/chicken_curry
there are 0 directories and 75 images in 10_food_classes_10_percent/train/fried_rice
there are 0 directories and 75 images in 10_food_classes_10_percent/train/ramen
there are 0 directories and 75 images in 10_food_classes_10_percent/train/pizza
there are 10 director

## Create data loaders (preparing the data)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32
EPOCHS = 5
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"
train_datagen = ImageDataGenerator(rescale=1/225.)
test_datagen = ImageDataGenerator(rescale=1/225.)
print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical"
                                             )


Training images:
Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"
train_data_10_percent_dataset = image_dataset_from_directory(train_dir,
                                                             label_mode="categorical",
                                                             batch_size=32,
                                                             image_size=(224,224),
                                                             shuffle=False

                                                             )
test_dataset = image_dataset_from_directory(test_dir,
                                           label_mode="categorical",
                                           batch_size=32,
                                           image_size=(224,224),
                                           shuffle=False)
print(train_data_10_percent_dataset,test_dataset)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))> <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>


In [ ]:
# normalizing training and validation datasets
rescale_layer = tf.keras.layers.Rescaling(1./225)
train_dataset_10_percent_rescaled = train_data_10_percent_dataset.map(lambda data,label:(rescale_layer(data),label))
test_dataset_rescaled = test_dataset.map(lambda data,label:(rescale_layer(data),label))
print(train_dataset_10_percent_rescaled)

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset_10_percent_rescaled = train_dataset_10_percent_rescaled.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
test_dataset_rescaled = test_dataset_rescaled.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical"),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomRotation(0.2)
])
train_dataset_10_percent_rescaled = train_dataset_10_percent_rescaled.map(lambda data,label:(data_augmentation(data,training=True),label))

In [ ]:
def create_model(model_url,num_classes):
  """
  Takes a Tensorflow Hub URL and creates a Keras Sequential model with it.

  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in the output layer,
    should be equeal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequentail model wtih model_url as feature extractor
    layer and Dense output layer with num_classes output neurons.

  """
  # Download the pretrained model and save it as keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE +(3,) # 3 for rgb color
                                           ) #freeze already learned patterns

  # Create our own model
  model = tf.keras.Sequential([
      feature_extractor_layer,
      tf.keras.layers.Dense(10,activation="softmax",name="output layer")
      ])

  return model


## Setting up callbacks (this to run whilst our model trains)
coallbacks are extra functionality you can add to your models to be performed during or after training. Some of the most popular coallbacks:
* Tracking experiments with the TensorBoard callbacks
* Model checkpoint with the ModelCheckpoint callbacks
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callbacks

In [ ]:
# Create tensorboard callback( functionized because we need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = os.path.join(dir_name,experiment_name,datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to : {log_dir}")
  return tensorboard_callback



### Creating and testing ResNet kaggle Hub Feature Extraction model

\## Creating a models using Kaggle Hub
In the past we've used TensorFlow to create our own models layer by layer from scratch.

Now we're going to do a similar process, except the majority of our model's layers are going to come from Kaggle Hub.
We can access pretrained models [here](https://www.kaggle.com/models/tensorflow/efficientnet/tensorFlow2/b0-feature-vector/1?tfhub-redirect=true&utm_source=chatgpt.com)

In [ ]:
!pip install tensorflow_hub
from tensorflow import keras
import tensorflow_hub as hub
class ResNetLayer(keras.layers.Layer):
    def __init__(self, resNet_feature_extractor_url, **kwargs):
        super().__init__(**kwargs)
        # create the hub layer inside init
        self.resNet_feature_extractor_url = resNet_feature_extractor_url
        self.resNet_feature_extractor = hub.KerasLayer(resNet_feature_extractor_url)

    def call(self, inputs):
        return self.resNet_feature_extractor(inputs)

    def get_config(self):
        config = super().get_config()
        config.update({
            "resNet_feature_extractor_url": self.resNet_feature_extractor_url
        })
        return config


resNet_feature_extractor = ResNetLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5")

resNet_model = tf.keras.Sequential([
    keras.Input(shape=(224,224,3)),
    resNet_feature_extractor,
    keras.layers.Dense(10,activation="softmax")
])
resNet_model.compile(loss="categorical_crossentropy",
                     optimizer="adam",
                     metrics=["accuracy"])
history_resNet_model = resNet_model.fit(train_dataset_10_percent_rescaled,
                                        epochs=5,
                                        steps_per_epoch=len(train_dataset_10_percent_rescaled),
                                        validation_data=test_dataset_rescaled,
                                        validation_steps=len(test_dataset_rescaled),
                                        callbacks=[create_tensorboard_callback("TesorFlow_Hub","resNet_model")]
                                        )


Saving TensorBoard log files to : TesorFlow_Hub/resNet_model/20251013-092348
Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.0802 - loss: 3.4266

KeyboardInterrupt: 

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
resNet_model.save("resNet_trained_model.h5")

In [ ]:
from google.colab import files
files.download("resNet_trained_model.h5")

In [ ]:
resNet_model.summary()

In [ ]:


# Step 3: Wrap the SavedModel in a Keras Layer
class EfficientNetLayer(keras.layers.Layer):
    def __init__(self, efficientNet_feature_extractor_url, **kwargs):
        super().__init__(**kwargs)
        # create the hub layer inside init
        self.efficientNet_feature_extractor_url = efficientNet_feature_extractor_url
        self.efficientNet_feature_extractor = hub.KerasLayer(efficientNet_feature_extractor_url)

    def call(self, inputs):
        return self.efficientNet_feature_extractor(inputs)

    def get_config(self):
        config = super().get_config()
        config.update({
            "efficientNet_feature_extractor_url": self.efficientNet_feature_extractor_url
        })
        return config

# Step 4: Build Sequential model
num_classes = 10

model = keras.Sequential([
    keras.Input(shape=(224, 224, 3)),  # input layer
    EfficientNetLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"),

    # keras.layers.Dropout(0.5),  # regularization
    keras.layers.Dense(num_classes, activation='softmax')  # classifier
])

model.layers[0].efficientNet_feature_extractor.trainable = False # Freeze already learned patterns
# Step 5: Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)




In [ ]:
model_weights = model.get_weights()
len(model_weights)


In [ ]:
efficientNet_history = model.fit(train_dataset_10_percent_rescaled,
          epochs=5,
          steps_per_epoch=len(train_dataset_10_percent_rescaled),
          validation_data=test_dataset_rescaled,
          validation_steps=len(test_dataset_rescaled),
          callbacks=[create_tensorboard_callback(dir_name="Tensorflow_Hub",experiment_name="EffiecientNet")]
          )

## Visualize the pre-trained model

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model,show_shapes=True,expand_nested=True,show_layer_names=True)

### Internal Archticture of EfficientNetBO Neural network

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

feature_extractor = EfficientNetB0(
    include_top=False, weights='imagenet', pooling='avg', input_shape=(224,224,3)
)
# feature_extractor.summary()


In [ ]:
# plot_model(feature_extractor,show_shapes=True,expand_nested=True)

In [ ]:
import tensorflow as tf

import tensorflow_hub as hub
num_classes = 10
m = tf.keras.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-feature-vector/1",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
m.build([None, 224, 224, 3])  # Batch input shape.
